In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv('../input/tabular-playground-series-apr-2021/train.csv')
df_test = pd.read_csv('../input/tabular-playground-series-apr-2021/test.csv')

In [ ]:
df_train['Fare']=df_train['Fare'].astype('float')

In [ ]:
df_test['Fare']=df_test['Fare'].astype('float')

In [ ]:
df_train.info()

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
df_train.isna().apply(pd.value_counts).T

In [ ]:
df_test.isna().apply(pd.value_counts).T

In [ ]:
df_train['Age'].fillna(df_train['Age'].median(),inplace=True)
df_test['Age'].fillna(df_test['Age'].median(),inplace=True)

In [ ]:
pd.set_option('display.max_rows',None)

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_train['Fare'].fillna(df_test['Fare'].median,inplace=True)
df_test['Fare'].fillna(df_test['Fare'].median,inplace=True)

In [ ]:
df_train['Ticket'].fillna('X',inplace=True)

In [ ]:
df_test['Ticket'].fillna('X',inplace=True)

In [ ]:
df_train.head()

In [ ]:
df_test['Cabin'].fillna('X',inplace=True)

In [ ]:
df_train['Cabin'].fillna('X',inplace=True)

In [ ]:
df_train['SibSp'].nunique()

In [ ]:
df_train['Embarked'].value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
df_train['Embarked'] = le.fit_transform(df_train['Embarked'])

In [ ]:
df_test['Embarked'] = le.fit_transform(df_test['Embarked'])

In [ ]:
df_test['Embarked'].fillna(df_test['Embarked'].median(),inplace=True)

In [ ]:
df_train['Embarked'].fillna(df_train['Embarked'].median(),inplace=True)

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
df_train['Sex'] = le.fit_transform(df_train['Sex']).astype('float')

In [ ]:
df_test['Sex'] = le.fit_transform(df_test['Sex']).astype('float')

In [ ]:
df_train['Ticket'] = le.fit_transform(df_train['Ticket'])
df_test['Ticket'] = le.fit_transform(df_test['Ticket'])

In [ ]:
df_train['Cabin'] = le.fit_transform(df_train['Cabin'])
df_test['Cabin'] = le.fit_transform(df_test['Cabin'])

In [ ]:
df_train.drop('Fare',axis=1,inplace=True)
df_test.drop('Fare',axis=1,inplace=True)

In [ ]:
df_train.head()

In [ ]:
df_train['Name'] = df_train['Name'].map(lambda x: x.split(',')[0])
df_test['Name'] = df_test['Name'].map(lambda x: x.split(',')[0])

In [ ]:
df_train['Name'] = le.fit_transform(df_train['Name']).astype('float')
df_test['Name'] = le.fit_transform(df_test['Name']).astype('float')

In [ ]:
X_train = df_train.drop(['PassengerId','Survived'],axis=1)
X_test = df_test.drop(['PassengerId'],axis=1)

In [ ]:
X_train.head()

In [ ]:
X_test.info()

In [ ]:
y_train = df_train['Survived']

In [ ]:
df_test.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
std = StandardScaler()

In [ ]:
X_test.head()

In [ ]:
X_train = std.fit_transform(X_train)
X_test = std.fit_transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
rf = RandomForestClassifier(n_estimators=500,criterion='gini',max_depth=7)

In [ ]:
rf.fit(X_train, y_train)
scores = cross_val_score(rf, X_train, y_train, cv=5)
scores.mean()

In [ ]:
from xgboost import XGBClassifier
xg = XGBClassifier()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
xg.fit(X_train,y_train)
scores_xg = cross_val_score(xg,X_train,y_train,cv=5)
scores_xg.mean()

In [ ]:
sample_submission = pd.read_csv('../input/tabular-playground-series-apr-2021/sample_submission.csv')

In [ ]:
predictions = rf.predict(X_test)
sample_submission['Survived'] = predictions
sample_submission.to_csv('submission.csv',index=False)

In [ ]:
predictions_xg = xg.predict(X_test)
sample_submission['Survived'] = predictions_xg
sample_submission.to_csv('submission_xg.csv',index=False)

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

clf = HistGradientBoostingClassifier()

clf.fit(X_train, y_train)
scores_clf = cross_val_score(clf, X_train, y_train, cv=5)
scores_clf.mean()

In [ ]:
predictions_clf = clf.predict(X_test)
sample_submission['Survived'] = predictions_clf
sample_submission.to_csv('submission_clf.csv',index=False)

In [ ]:
from sklearn.ensemble import BaggingClassifier
bagging = BaggingClassifier()
bagging.fit(X_train,y_train)
scores_bagging = cross_val_score(bagging,X_train,y_train,cv=5)
scores_bagging.mean()

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='lbfgs')
lr.fit(X_train,y_train)
scores_lr = cross_val_score(bagging,X_train,y_train,cv=5)
scores_lr.mean()

In [ ]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

In [ ]:
N_ESTIMATORS = 500
N_SPLITS = 10
SEED = 2021
EARLY_STOPPING_ROUNDS = 100
VERBOSE = False

In [ ]:
parameters = {
    'max_depth': np.arange(2, 5, dtype=int),
    'min_samples_leaf':  np.arange(2, 5, dtype=int),
}

In [ ]:
lgbm_params = {
    'metric': 'binary_logloss',
    'n_estimators': N_ESTIMATORS,
    'objective': 'binary',
    'random_state': SEED,
    'learning_rate': 0.01,
    'min_child_samples': 150,
    'reg_alpha': 3e-5,
    'reg_lambda': 9e-2,
    'num_leaves': 20,
    'max_depth': 16,
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'subsample_freq': 2,
    'max_bin': 240,
}

In [ ]:
lgbm = LGBMClassifier(**lgbm_params)
lgbm.fit(X_train,y_train)
scores_lgbm = cross_val_score(lgbm,X_train,y_train,cv=5)
scores_lgbm.mean()

In [ ]:
predictions_lgbm = lgbm.predict(X_test)
sample_submission['Survived'] = predictions_lgbm
sample_submission.to_csv('submission_lgbm.csv',index=False)